In [5]:
import pandas as pd

In [8]:
# Load datasets
df1 = pd.read_csv('/Users/HP/Desktop/QTM498R/EPD Dataset/EPDLakesSounds.csv')
df2 = pd.read_csv('/Users/HP/Desktop/QTM498R/EPD Dataset/EPDStreamsBeaches.csv')

# Common columns to keep
columns_to_keep = ["ReachID", "Name", "County", "WaterType", "Uses", "Assessment", "Category", "Cause"]

# Filter relevant columns
df1_filtered = df1[columns_to_keep]
df2_filtered = df2[columns_to_keep]

# Merge datasets
merged_df = pd.concat([df1_filtered, df2_filtered], ignore_index=True)
display(merged_df)

,ReachID,Name,County,WaterType,Uses,Assessment,Category,Cause
0,GAR031300010516,Lake Qualatchee,White,Lake,Fishing,Supporting,1,
1,GAR060200030118,Winfield Scott Lake,Union,Lake,Fishing,Supporting,1,
2,GAR031501080909,John Tanner Lake (upper lake),Carroll,Lake,Fishing,Supporting,1,
3,GAR031300040203,Yohola Lake,Early,Lake,Fishing,Assessment Pending,3,
4,GAR030701030211,Shamrock Lake,Clayton,Lake,Fishing,Supporting,1,
...,...,...,...,...,...,...,...,...
3088,GAR060102020104,Mud Creek,Rabun,Stream,"Drinking Water, Fishing",Assessment Pending,2,
3089,GAR060102020101,Little Tennessee River,Rabun,Stream,Fishing,Not Supporting,4a,Bacteria
3090,GAR030702010305,Tributary to Little Red Bluff Creek,Atkinson,Stream,Fishing,Assessment Pending,3,
3091,GAR031102020308,Hat Creek,Turner,Stream,Fishing,Assessment Pending,3N,


In [9]:
# Map the distinct categories
unique_category = merged_df['Category'].unique()
display(unique_category)

category_mapping = {
    '1': 'Complied',
    '2': 'Pending',
    '3': 'Pending',
    '3N': 'Pending',
    '5': 'Not_Complied',
    '5R': 'Not_Complied',
    '4a': 'Not_Complied',
    '4b': 'Not_Complied',
    '4C': 'Not_Complied'
}

array(['1', '3', '5', '4a', '4a, 5', '2', '5, 5R', '4c', '3, 3N', '3N',
       '4b', '4a, 4b'], dtype=object)

In [10]:
# Map the original dataset by categorizing the categories to different situations
def map_category(value):
    # Split by comma and strip spaces
    categories = value.split(', ')
    # Map each category to its new value
    mapped_values = [category_mapping.get(cat, 'Unknown') for cat in categories]
    # Take the first mapped value, or join if multiple
    return mapped_values[0] if len(mapped_values) == 1 else ', '.join(set(mapped_values))

# Apply function to handle multi-category values
merged_df['Compliance'] = merged_df['Category'].apply(map_category)
display(merged_df)

,ReachID,Name,County,WaterType,Uses,Assessment,Category,Cause,Compliance
0,GAR031300010516,Lake Qualatchee,White,Lake,Fishing,Supporting,1,,Complied
1,GAR060200030118,Winfield Scott Lake,Union,Lake,Fishing,Supporting,1,,Complied
2,GAR031501080909,John Tanner Lake (upper lake),Carroll,Lake,Fishing,Supporting,1,,Complied
3,GAR031300040203,Yohola Lake,Early,Lake,Fishing,Assessment Pending,3,,Pending
4,GAR030701030211,Shamrock Lake,Clayton,Lake,Fishing,Supporting,1,,Complied
...,...,...,...,...,...,...,...,...,...
3088,GAR060102020104,Mud Creek,Rabun,Stream,"Drinking Water, Fishing",Assessment Pending,2,,Pending
3089,GAR060102020101,Little Tennessee River,Rabun,Stream,Fishing,Not Supporting,4a,Bacteria,Not_Complied
3090,GAR030702010305,Tributary to Little Red Bluff Creek,Atkinson,Stream,Fishing,Assessment Pending,3,,Pending
3091,GAR031102020308,Hat Creek,Turner,Stream,Fishing,Assessment Pending,3N,,Pending


In [11]:
# Some of the water belongs to multiple counties, so I divide them into each of the county to make the compliance ratio
# Function to expand multi-county rows and count each compliance type correctly
def expand_counties(row):
    counties = row['County'].split(', ')  # Split multi-county names
    compliance = row['Compliance']
    
    return [{
        'County': county,
        'Complied': 1 if compliance == 'Complied' else 0,
        'Not_Complied': 1 if compliance == 'Not_Complied' else 0,
        'Pending': 1 if compliance == 'Pending' else 0,
        'Total': 1  # Every county gets counted once per water body
    } for county in counties]

# Create dataframe for these ratios
expanded_data = []
for _, row in merged_df.iterrows():
    expanded_data.extend(expand_counties(row))
expanded_df = pd.DataFrame(expanded_data)
compliance_counts = expanded_df.groupby('County').sum()

# Compute ratios
compliance_counts['Compliance_Ratio'] = compliance_counts['Complied'] / compliance_counts['Total']
compliance_counts['Non_Compliance_Ratio'] = compliance_counts['Not_Complied'] / compliance_counts['Total']
compliance_counts['Pending_Ratio'] = compliance_counts['Pending'] / compliance_counts['Total']

compliance_ratio_df = compliance_counts.reset_index()[['County', 'Compliance_Ratio', 'Non_Compliance_Ratio', 'Pending_Ratio']]
print(compliance_ratio_df)

        County  Compliance_Ratio  Non_Compliance_Ratio  Pending_Ratio
0      Appling          0.100000              0.500000       0.400000
1     Atkinson          0.250000              0.500000       0.250000
2        Bacon          0.200000              0.400000       0.400000
3        Baker          0.500000              0.500000       0.000000
4      Baldwin          0.777778              0.166667       0.055556
..         ...               ...                   ...            ...
154  Whitfield          0.250000              0.722222       0.027778
155     Wilcox          0.083333              0.750000       0.166667
156     Wilkes          0.541667              0.416667       0.041667
157  Wilkinson          0.538462              0.384615       0.076923
158      Worth          0.153846              0.692308       0.153846

[159 rows x 4 columns]


In [12]:
# To csvs
merged_df.to_csv("/Users/HP/Desktop/QTM498R/EPD Dataset/Merged Datasets.csv", index=False)
compliance_ratio_df.to_csv("/Users/HP/Desktop/QTM498R/EPD Dataset/Compliance Ratio.csv", index=False)
